In [1]:
#all imports
from tensorflow import keras
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator
from keras._tf_keras.keras.optimizers import Adam
from keras._tf_keras.keras.preprocessing.image import img_to_array
from keras._tf_keras.keras.utils import to_categorical, plot_model
from keras._tf_keras.keras.models import Sequential
from keras._tf_keras.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from keras import backend as K
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os
import glob

In [2]:
epochs = 100 #how many times it reads through is what epochs is. 
lr = 1e-3 #learning rate
batch_size = 64 #how many images to look at a time
img_dims = (96,96,3) # Height, width, num of RGB channels

In [14]:
data = []
labels = []
#arrays for data and labels 
image_files = [f for f in glob.glob(r'/Users/shiva/Downloads/AI - OPENCV - ASL_DETECTOR/asl_alphabet_train/' + "/**/*", recursive=True) if not os.path.isdir(f)]
#takes the image files from the files and then put it in the arrays
random.shuffle(image_files) #shuffling the image files. 
#glob.glob is like a library that reads in data files and searching for files that follow a data pattern. 

In [15]:
# Adds images to array, also gives it a label
for img in image_files:
    image = cv2.imread(img)#look at image
    image = cv2.resize(image, (img_dims[0], img_dims[1])) # Resizes every image to be the same size. 
    image = img_to_array(image)#convert to array
    data.append(image)#add to data list 

    label = img.split(os.path.sep)[-2]#add label to each image classifying what the alphabet is for each image. 
    label = ord(label) - ord('A') # Convert label to numerical value (0-25)
    labels.append([label])# actually add to data list. 

# Data pre-processing, converts data and labels to numpy array (Easier for computer to work with numpy arrays instead of normal arrays. )
try:
    data = np.array(data, dtype='float') / 255.0 
    labels = np.array(labels)
except Exception as e:
    print(f"Error converting data to numpy array: {e}")

In [17]:
# Splits dataset into training and testing
# test_size = 0.2 means that 20% of the dataset will be used for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size = 0.2, random_state=42)

trainY = to_categorical(trainY, num_classes=26)#we have 26 folders for 26 aplhabets. 
testY = to_categorical(testY, num_classes=26)

# Augmentation of dataset (Changes and adds error and adds altered version of the training data.)
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1, height_shift_range=0.1, 
                         shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode="nearest")